###  K-means with Euclidena distance as proximity measure

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random as rd

In [ ]:
def getEucledian(a,b):
    #print(a,b)
    total = 0;
    for i in range(0,len(a)):
        diff = b[i] - a[i];
        total += diff * diff;
    return np.sqrt(total)

In [ ]:
def normContinuousAttributes(df,labels):
    for i in labels:
        df[i] = df[[i]].apply(lambda x : (x - np.min(x))/(np.max(x) - np.min(x)))
    return df

In [ ]:
def clusterSSE(x,k_clusters,k_centers):
    total = {} 
    for key,value in k_clusters.items():
        s = 0
        center = k_centers[key]
        for i in value:
            s = s + math.pow(getEucledian(x[i],center),2)
        total[key] = s
    return total       

In [ ]:
def overallSSE(cluster_sse):
    s = 0
    for key,value in cluster_sse.items():
        s = s + cluster_sse[key]
    return s

In [ ]:
def SSB(x,k_clusters,k_centers):
    c = getMeanUsingIndex(x,list(range(0,len(x))))
    keys = list(k_centers.keys())
    s =  0
    for j in keys:
        t = len(k_clusters[j])
        s = s + (t * math.pow(getEucledian(k_centers[j],c),2))
    return s

In [ ]:
def getMeanUsingIndex(x,indices):
    total = []
    for i in x[0]:
        total.append(0)
    #print(total)
    for i in indices:
        for j in range(0,len(total)):
            total[j] = total[j] + x[i][j]
    total = [f/len(indices) for f in total]
    return (total)

In [ ]:
def anyCenterChanged(k_centers,new_centers):
    for key,value in k_centers.items():
        if(k_centers[key] != new_centers[key]):
            return True
    return False

In [ ]:
def assignInputToClusters(x,k_centers):
    #keep track of clusters
    k_clusters = {}
    # print("Centers : ")
    # print(k_centers)
    dissimilarity = []
    #assign all input patterns to clusters
    for key,i in k_centers.items():
        for j in x:
            dissimilarity.append(getEucledian(i,j)) #eucledian similarity

    dissimilarity = np.reshape(dissimilarity,(len(k_centers),len(x)))
    df_dissim = pd.DataFrame(data=dissimilarity)
    #print(df_dissim)
    for i in range(1,df_dissim.shape[0]+1):
        k_clusters[i] = [];
    for i in df_dissim.columns:
        #print(df_dissim.nsmallest(1,i).index[0],end = " ")
        shortest_dist_index = df_dissim.nsmallest(1,i).index[0]
        k_clusters[shortest_dist_index+1].append(i)
    #print(k_clusters)
    new_centers = {}
    for key,value in k_clusters.items():
        #print(key,value)
        new_centers[key]  = getMeanUsingIndex(x,value)
#     print("New Centers : ")
#     print(new_centers)
    if(anyCenterChanged(k_centers,new_centers)):
        assignInputToClusters(x,new_centers)
    return (new_centers,k_clusters)

In [ ]:
def recomputeDist(x,centers):
    dist = []
    for i in x:
        t = []
        for key,value in centers.items():
            t.append(getEucledian(i,value))
        dist.append(np.min(t))
    total = np.sum(dist)
    prob_dist = [d/total for d in dist]
    return dist,prob_dist

In [ ]:
def kMeansPlusPlus(x,K,first_center):
    k_centers = {}
    #randomly select one center
    #k_centers[1] = rd.sample(x,1)[0]
    k_centers[1] = first_center
    #print("First center : " + str(k_centers[1]))
    #calculate distance of all points to this center
    dist = []
    for i in x:
        dist.append(math.pow(getEucledian(i,k_centers[1]),2))
    total = np.sum(dist)
    #get probability distribution for each point using the distance
    prob_dist = [d/total for d in dist]
    #select k centers using probability distribution
    j = 2
    index = list(range(0,len(x)))
    while(not(len(k_centers) == K)):
        i = np.random.choice(index,p=prob_dist)
        k_centers[j] = x[i]
        dist,prob_dist = recomputeDist(x,k_centers)
        j = j + 1
    return k_centers
    

In [ ]:
def kMeans(x,K,kmeans_plus_plus,first_center):
    #select K number of input patterns as cluster centers randomly
    centers = rd.sample(x,K)
    k_centers = {}
    if(kmeans_plus_plus):
        k_centers = kMeansPlusPlus(x,K,first_center)
    else:
        k_centers = {}
        i = 1
        for j in centers:
            k_centers[i] = j
            i = i + 1
    (k_centers,k_clusters) = assignInputToClusters(x,k_centers)
    return (k_clusters,k_centers)


In [ ]:
def getOutFrame(k_clusters):
    columns = ["Row ID","Cluster"]
    df_out = pd.DataFrame() 
    for key,values in k_clusters.items():
        t = []
        for i in values:
            t.append(key)
        df = pd.DataFrame({"Row ID" : values, "Cluster" : t})
        df_out = df_out.append(df)
        df_out = df_out.sort_values("Row ID",ascending=True)
#     indices = df_out["Row ID"]
#     indices = [x+1 for x in indices]
#     df_out["Row ID"] = indices
    return df_out

In [ ]:
def plotClusters(x,k_clusters,title,xlabel,ylabel):
    fig1 = plt.figure(1)
    for key,value in k_clusters.items():
        c = [x[index] for index in value]
        #print(c)
        plt.scatter(*zip(*c),label="Cluster " + str(key))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
def getTrueClusters(df,label):
    k_clusters = {}
    k_centers = {}
    clusters = set(df[label])
    #print(clusters)
    df_temp = df.drop(["ID",label],axis=1)
    x = df_temp.values.tolist()
    for i in clusters:
        a = df.loc[df[label] == i].index
        #print(a)
        k_clusters[i] = list(a)
        k_centers[i] = getMeanUsingIndex(x,list(a))
    #print(k_clusters)
    return k_clusters,k_centers

In [ ]:
def crossTabMatrix(k_clusters,true_clusters):
    df = pd.DataFrame()
    t = []
    for key,value in k_clusters.items():
        for a,b in true_clusters.items():
            b3 = [val for val in value if val in b]
            t.append(len(b3))
    t = np.reshape(t,(len(k_clusters),len(true_clusters)))
    columns = list(true_clusters.keys())
    df_temp = pd.DataFrame(data=list(k_clusters.keys()),columns=["Cluster"])
    df = pd.DataFrame(data=t,columns=columns)
    df = pd.concat([df_temp,df],axis=1)
    df = df.set_index('Cluster')
    return df

### Read TwoDimHard.csv

In [ ]:
def readTwoDimHard(file):
    df_tdh = pd.read_csv("TwoDimHard.csv")
    true_clusters,true_centers = getTrueClusters(df_tdh,"cluster")
    df_tdh = df_tdh.drop(["ID","cluster"],axis=1)
    x = df_tdh.values.tolist()
    return x,true_clusters,true_centers

### Read Wine.csv

In [ ]:
def readWine(file):
    df_wine = pd.read_csv("wine.csv")
    df_wine = df_wine.drop(["class"],axis=1)
    labels = ["fx_acidity","resid_sugar","free_sulf_d","tot_sulf_d","pH","alcohol"]
    df_wine = normContinuousAttributes(df_wine,labels)
    true_clusters,true_centers = getTrueClusters(df_wine,"quality")
    df_wine = df_wine.drop(["ID","quality"],axis=1)
    x = df_wine.values.tolist()
    return x,true_clusters,true_centers

### To run K-means for wine dataset, assign True to wine variable. Otherwise TwoDimHard will be used

In [ ]:
wine = False
if(wine):
    x,true_clusters,true_centers = readWine("wine.csv")
else:
    x,true_clusters,true_centers = readTwoDimHard("TwoDimHard.csv")

In [ ]:
# wine = True
# df_tdh = pd.read_csv("wine.csv")
# df_tdh = df_tdh.drop(["class"],axis=1)
# #df_tdh['quality'] = df_tdh['quality'].map({3:1,4:2,5:3,6:4,7:5,8:6})
# labels = ["fx_acidity","resid_sugar","free_sulf_d","tot_sulf_d","pH","alcohol"]
# df_tdh = normContinuousAttributes(df_tdh,labels)
# true_clusters,true_centers = getTrueClusters(df_tdh,"quality")
# df_tdh = df_tdh.drop(["ID","quality"],axis=1)
# x = df_tdh.values.tolist()

### True Cluster SSE and SSB

In [ ]:
true_sse = clusterSSE(x,true_clusters,true_centers)
print("True Cluster SSE : " + str(true_sse))
true_overall_sse = overallSSE(true_sse)
print("True Overall SSE : " + str(true_overall_sse))
true_ssb = SSB(x,true_clusters,true_centers)
print("True SSB : " + str(true_ssb))

### Input k

In [ ]:
#first_center = rd.sample(x,1)[0]
k = int(input("Enter K : "))
t = []
t1 = []
k_clusters =  {}
k_centers = {}
best_centers = {}
best_sse = 300
best_ssb = 0
columns = ["Cluster","Overall SSE","SSB"]
#df_table = pd.DataFrame({"Cluster" : [], "Overall SSE" : [], "Overall SSB" : []})
df_table = pd.DataFrame()

### n1 = 0 and n2 = 1 : Run K-means once. n1 = 0 and n2 = 10 : Run K-means 10 times

In [ ]:
n1 = 0
n2 = 1
first_center = rd.sample(x,1)[0]
for i in range(n1,n2):
    #first_center = rd.sample(x,1)[0]
    print("Iteration : " + str(i))
    (k_clusters,k_centers) = kMeans(x,k,False,first_center)
    #print(k_centers)
    sse = clusterSSE(x,k_clusters,k_centers)
    print("Cluster SSE : " + str(sse))
    overall_sse = overallSSE(sse)
    print("Overall SSE : " + str(overall_sse))
    t.append(overall_sse)
    ssb = SSB(x,k_clusters,k_centers)
    print("SSB : " + str(ssb))
    t1.append(ssb)
    if(overall_sse > best_sse ):
        best_sse = overall_sse
        global best_clusters
        best_clusters = k_clusters
        global best_centers
        best_centers = k_centers
        best_ssb = ssb
        global best_cluster_sse
        best_cluster_sse = sse
    df = pd.DataFrame({"Cluster" : "Assigned Cluster", "Overall SSE" : [overall_sse],"SSB" : [ssb]})
    df_table = df_table.append(df) 
    df = pd.DataFrame({"Cluster" : "True Cluster", "Overall SSE" : [true_overall_sse],"SSB" : [true_ssb]})
    df_table = df_table.append(df)
    df_table = df_table.set_index('Cluster')
df_cross = crossTabMatrix(k_clusters,true_clusters)
df_out = getOutFrame(k_clusters)
if(wine):
    df_out.to_csv("wine_output.csv")
else:
    df_out.to_csv("TwoDimHard_output.csv")

In [ ]:
df_cross.head()

In [ ]:
#df_table.head()

In [ ]:
# best_overall_sse = np.min(t)
# print("Overall SSE's : " + str(t))
# print("Best SSE : " + str(best_overall_sse))
# print("Average SSE : " + str(np.average(t)))
# print("Worst SSE : " + str(np.max(t)))
# best_overall_ssb = np.min(t1)
# print("Overall SSB's : " + str(t1))
# print("Best SSB : " + str(best_overall_ssb))
# print("Average SSB : " + str(np.average(t1)))
# print("Worst SSB : " + str(np.max(t1)))

In [ ]:
#plt.subplot(1,1,1)
#plotClusters(x,k_clusters,"Clusters with K = " + str(k),"x1","x2")
#plt.subplot(1,1,2)
#plotClusters(x,true_clusters,"True Clusters with K = " + str(len(true_centers)),"x1","x2")
df_out.head()

In [ ]:
def plot(y,k_list,ylabel,title):
    my_xticks = k_list
    s = list(range(0,len(k_list)))
    plt.xticks(s, my_xticks)
    print(s,y)
    plt.plot(s,y)
    plt.xlabel("k")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [ ]:
# print(best_sse)
# print(best_cluster_sse)
# print(best_ssb)

### For Wine Dataset if varying k from 2 to 11

In [ ]:
if(wine):
    plot(t,list(range(n1,n2)),"Overall SSE","Overall SSE vs K")
    plot(t1,list(range(n1,n2)),"SSB","SSB vs K")